## COMPUTER VISION: A Deep Learning Model for Ultrasound Imaging in Non-Destructive Testing ##

#### Author: My Lan Nguyen, Harry Hoang, Donald Nguyen 

**1. Introduction**
Ultrasound-based Non Destructive Testing (NDT) is one of the highly popular 

In order to address the question above, we will explore two real-world datasets: training and testing from DarkVision. 

In [1]:
!pip install tensorflow

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import trimesh
import os
from scipy.ndimage import zoom
import gc 
import gc
import numpy as np

In [12]:
class VolumetricToMeshModel(tf.keras.Model):
    def __init__(self, latent_dim=1024, num_vertices=10000):
        super(VolumetricToMeshModel, self).__init__()

        # 3D-CNN Backbone
        self.backbone = models.Sequential([
            layers.Conv3D(16, kernel_size=3, strides=1, padding="same", activation="relu"),
            layers.BatchNormalization(),
            layers.MaxPooling3D(pool_size=2, strides=2),

            layers.Conv3D(32, kernel_size=3, strides=1, padding="same", activation="relu"),
            layers.BatchNormalization(),
            layers.MaxPooling3D(pool_size=2, strides=2),

            layers.Conv3D(64, kernel_size=3, strides=1, padding="same", activation="relu"),
            layers.BatchNormalization(),
            layers.MaxPooling3D(pool_size=2, strides=2),

            layers.Conv3D(128, kernel_size=3, strides=1, padding="same", activation="relu"),
            layers.BatchNormalization(),
            layers.GlobalAveragePooling3D()
        ])

        # Fully connected layers for regression
        self.fc = models.Sequential([
            layers.Dense(latent_dim, activation="relu"),
            layers.Dense(num_vertices * 3)  # Predict x, y, z for each vertex
        ])

    def call(self, inputs):
        x = self.backbone(inputs)
        x = self.fc(x)
        return tf.reshape(x, (-1, tf.shape(x)[1] // 3, 3))  # Output shape: (batch_size, num_vertices, 3)


In [13]:
class VolumetricMeshDataset(tf.data.Dataset):
    def __new__(cls, volumes_dir, meshes_dir, input_shape=(128, 128, 128), num_vertices=10000):
        # Get file paths
        volume_files = sorted([os.path.join(volumes_dir, f) for f in os.listdir(volumes_dir) if f.endswith('.raw')])
        mesh_files = sorted([os.path.join(meshes_dir, f) for f in os.listdir(meshes_dir) if f.endswith('.ply')])

        def preprocess(volume_path, mesh_path):
            # Load and normalize volume
            with open(volume_path, 'rb') as f:
                volume = np.frombuffer(f.read(), dtype=np.uint16).reshape((768, 768, 1280))
            volume = volume / np.max(volume)  # Normalize to [0, 1]
            
            # Resize volume to input_shape
            zoom_factors = [input_shape[0] / volume.shape[0], 
                            input_shape[1] / volume.shape[1], 
                            input_shape[2] / volume.shape[2]]
            volume = zoom(volume, zoom_factors, order=1)  # Bilinear interpolation
            volume = tf.expand_dims(volume, axis=-1)  # Add channel dimension

            # Load mesh
            mesh = trimesh.load(mesh_path, process=False)
            vertices = mesh.vertices.astype(np.float32)
            vertices = vertices - np.mean(vertices, axis=0)  # Center vertices
            vertices = vertices / np.max(np.linalg.norm(vertices, axis=1))  # Normalize to unit sphere

            # Pad vertices if fewer than num_vertices
            if vertices.shape[0] < num_vertices:
                padding = np.zeros((num_vertices - vertices.shape[0], 3), dtype=np.float32)
                vertices = np.vstack([vertices, padding])
            else:
                vertices = vertices[:num_vertices]

            return volume, vertices

        def generator():
            for vol_path, mesh_path in zip(volume_files, mesh_files):
                yield preprocess(vol_path, mesh_path)

        dataset = tf.data.Dataset.from_generator(
            generator,
            output_signature=(
                tf.TensorSpec(shape=(input_shape[0], input_shape[1], input_shape[2], 1), dtype=tf.float32),
                tf.TensorSpec(shape=(num_vertices, 3), dtype=tf.float32)
            )
        )

        return dataset

In [15]:
if __name__ == "__main__": 
    volumes_dir = "training/volumes"
    meshes_dir = "training/meshes"

    dataset = VolumetricMeshDataset(volumes_dir, meshes_dir)
    train_size = int(0.8 * len(list(dataset)))
    test_size = len(list(dataset)) - train_size

    train_dataset = dataset.take(train_size).batch(2).prefetch(tf.data.AUTOTUNE)
    test_dataset = dataset.skip(train_size).batch(2).prefetch(tf.data.AUTOTUNE)

    # Create model
    model = VolumetricToMeshModel()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=["mse"])

    # Fit the model
    model.fit(train_dataset, epochs=1, validation_data=test_dataset)
    model.save("source/volumetric_to_mesh_model.keras")

2/2 ━━━━━━━━━━━━━━━━━━━━ 37s 28s/step - loss: 0.1112 - mse: 0.1112 - val_loss: 0.1484 - val_mse: 0.1484


In [16]:
# Chamfer Distance Function
def chamfer_distance(predicted, ground_truth):
    """
    Computes the Chamfer Distance between two point clouds.
    """
    dists_pred_to_gt = tf.reduce_min(
        tf.reduce_sum((tf.expand_dims(predicted, axis=1) - tf.expand_dims(ground_truth, axis=0))**2, axis=-1), axis=1
    )
    dists_gt_to_pred = tf.reduce_min(
        tf.reduce_sum((tf.expand_dims(ground_truth, axis=1) - tf.expand_dims(predicted, axis=0))**2, axis=-1), axis=1
    )
    chamfer = tf.reduce_mean(dists_pred_to_gt) + tf.reduce_mean(dists_gt_to_pred)
    return chamfer.numpy()

In [17]:
# Hausdorff Distance Function
def hausdorff_distance(predicted, ground_truth):
    """
    Computes the Hausdorff Distance between two point clouds.
    """
    dists_pred_to_gt = tf.reduce_min(
        tf.reduce_sum((tf.expand_dims(predicted, axis=1) - tf.expand_dims(ground_truth, axis=0))**2, axis=-1), axis=1
    )
    dists_gt_to_pred = tf.reduce_min(
        tf.reduce_sum((tf.expand_dims(ground_truth, axis=1) - tf.expand_dims(predicted, axis=0))**2, axis=-1), axis=1
    )
    hausdorff = max(tf.reduce_max(dists_pred_to_gt).numpy(), tf.reduce_max(dists_gt_to_pred).numpy())
    return hausdorff

In [18]:
class VolumetricToMeshModel(tf.keras.Model):
    def __init__(self, latent_dim=1024, num_vertices=10000, **kwargs):
        super(VolumetricToMeshModel, self).__init__(**kwargs)
        self.latent_dim = latent_dim
        self.num_vertices = num_vertices

        self.backbone = tf.keras.Sequential([
            tf.keras.layers.Conv3D(16, kernel_size=3, strides=1, padding="same", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling3D(pool_size=2, strides=2),

            tf.keras.layers.Conv3D(32, kernel_size=3, strides=1, padding="same", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling3D(pool_size=2, strides=2),

            tf.keras.layers.Conv3D(64, kernel_size=3, strides=1, padding="same", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling3D(pool_size=2, strides=2),

            tf.keras.layers.Conv3D(128, kernel_size=3, strides=1, padding="same", activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.GlobalAveragePooling3D()
        ])

        self.fc = tf.keras.Sequential([
            tf.keras.layers.Dense(latent_dim, activation="relu"),
            tf.keras.layers.Dense(num_vertices * 3)
        ])

    def call(self, inputs):
        x = self.backbone(inputs)
        x = self.fc(x)
        return tf.reshape(x, (-1, tf.shape(x)[1] // 3, 3))

    def get_config(self):
        config = super(VolumetricToMeshModel, self).get_config()
        config.update({
            "latent_dim": self.latent_dim,
            "num_vertices": self.num_vertices
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [19]:
# Load the saved model
model = tf.keras.models.load_model(
    "source/volumetric_to_mesh_model.keras",
    custom_objects={"VolumetricToMeshModel": VolumetricToMeshModel}
)

In [20]:
# Testing data directories
test_volumes_dir = "testing/volumes"
test_meshes_dir = "testing/meshes"

In [21]:
# Prepare test dataset
def preprocess(volume_path, mesh_path):
    with open(volume_path, 'rb') as f:
        volume = np.frombuffer(f.read(), dtype=np.uint16).reshape((768, 768, 1280))
    volume = volume / np.max(volume)
    zoom_factors = [128 / volume.shape[0], 128 / volume.shape[1], 128 / volume.shape[2]]
    volume = zoom(volume, zoom_factors, order=1)
    volume = tf.expand_dims(volume, axis=-1)
    mesh = trimesh.load(mesh_path, process=False)
    vertices = mesh.vertices.astype(np.float32)
    vertices = vertices - np.mean(vertices, axis=0)
    vertices = vertices / np.max(np.linalg.norm(vertices, axis=1))
    return volume, vertices

def test_generator():
    volume_files = sorted([os.path.join(test_volumes_dir, f) for f in os.listdir(test_volumes_dir) if f.endswith('.raw')])
    mesh_files = sorted([os.path.join(test_meshes_dir, f) for f in os.listdir(test_meshes_dir) if f.endswith('.ply')])
    for vol_path, mesh_path in zip(volume_files, mesh_files):
        yield preprocess(vol_path, mesh_path)

test_dataset = tf.data.Dataset.from_generator(
    test_generator,
    output_signature=(
        tf.TensorSpec(shape=(128, 128, 128, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 3), dtype=tf.float32)
    )
).batch(1)

In [31]:
# Evaluate metrics for file 1
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Average Chamfer Distance: 1.3098429441452026


In [30]:
# Evaluate metrics for file 2
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Average Chamfer Distance: 0.9633491635322571


In [29]:
# Evaluate metrics for file 3
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 795ms/step
Average Chamfer Distance: 0.8123236894607544


In [28]:
# Evaluate metrics for file 4
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 872ms/step
Average Chamfer Distance: 0.44448643922805786


In [22]:
# Evaluate metrics for file 5
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
Average Chamfer Distance: 0.3655959367752075


In [23]:
# Evaluate metrics for file 6
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
Average Chamfer Distance: 0.45561835169792175


In [24]:
# Evaluate metrics for file 7
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Average Chamfer Distance: 0.46415770053863525


In [25]:
# Evaluate metrics for file 8
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Average Chamfer Distance: 0.3692156970500946


In [26]:
# Evaluate metrics for file 9
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Average Chamfer Distance: 0.4162712097167969


In [27]:
# Evaluate metrics for file 10
chamfer_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        chamfer_scores.append(chamfer_distance(predicted, ground_truth))

# Calculate average scores
avg_chamfer = np.mean(chamfer_scores)
print(f"Average Chamfer Distance: {avg_chamfer}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 843ms/step
Average Chamfer Distance: 0.47973278164863586


In [32]:
# Evaluate metrics for file 1
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
Average Hausdorff Distance: 0.9990211129188538


In [33]:
# Evaluate metrics for file 2
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Average Hausdorff Distance: 0.9987767338752747


In [34]:
# Evaluate metrics for file 3
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Average Hausdorff Distance: 0.9989096522331238


In [ ]:
# Evaluate metrics for file 4
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [35]:
# Evaluate metrics for file 5
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
Average Hausdorff Distance: 0.9987020492553711


In [36]:
# Evaluate metrics for file 6
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
Average Hausdorff Distance: 0.9987960457801819


In [37]:
# Evaluate metrics for file 7
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
Average Hausdorff Distance: 0.9985036849975586


In [38]:
# Evaluate metrics for file 8
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Average Hausdorff Distance: 0.9987719058990479


In [39]:
# Evaluate metrics for file 9
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Average Hausdorff Distance: 0.998645544052124


In [40]:
# Evaluate metrics for file 10
hausdorff_scores = []

for volume_batch, gt_mesh_batch in test_dataset:
    predicted_mesh_batch = model.predict(volume_batch)
    for predicted, ground_truth in zip(predicted_mesh_batch, gt_mesh_batch):
        hausdorff_scores.append(hausdorff_distance(predicted, ground_truth))

# Calculate average scores
avg_hausdorff = np.mean(hausdorff_scores)
print(f"Average Hausdorff Distance: {avg_hausdorff}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 842ms/step
Average Hausdorff Distance: 0.999098539352417
